# Overview of the Data

In [2]:
%%capture
%load_ext sql
%sql sqlite:///Datasets/chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///Datasets/chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Highest Selling Genres in USA

In [4]:
%%sql

WITH usa_customers AS(
    SELECT * FROM invoice
    WHERE billing_country = 'USA')


SELECT
    g.name 'genre',
    SUM(il.quantity) 'total_sales',
    (CAST(SUM(il.quantity) AS Float) / (SELECT SUM(il.quantity) FROM invoice_line il
                                        INNER JOIN usa_customers uc ON uc.invoice_id = il.invoice_id)) * 100 'sale_percentage'
FROM invoice_line il
INNER JOIN usa_customers uc ON uc.invoice_id = il.invoice_id
LEFT JOIN track t ON t.track_id = il.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY g.genre_id
ORDER BY total_sales DESC

 * sqlite:///Datasets/chinook.db
Done.


genre,total_sales,sale_percentage
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.425309229305423
Alternative,35,3.3301617507136063
Pop,22,2.093244529019981
Latin,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


The output of this query reveals that Rock albums are significantly superior to all other genres in sails, making this the most suitable choice of genre if a record label were to sign an artist. Punk and Metal artists may also be considered.

## Analysing Employee Sales Performance
Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. I will be analysing the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [26]:
%%sql

WITH
    supervising AS(
        SELECT
            e1.employee_id 'employee_id',
            e1.last_name 'employee_last_name',
            e1.first_name 'employee_first_name',
            e1.reports_to 'reports_to_id',
            e2.first_name 'supervisor_first_name',
            e2.last_name 'supervisor_last_name'
        FROM employee e1
        LEFT JOIN employee e2 ON e1.reports_to = e2.employee_id
),

    employee_totals AS(
        SELECT
            e.employee_id,
            e.last_name,
            ROUND(SUM(i.total), 2) 'total'
        FROM invoice i
        INNER JOIN customer c ON c.customer_id = i.customer_id
        INNER JOIN employee e ON e.employee_id = c.support_rep_id
        WHERE title = 'Sales Support Agent'
        GROUP BY c.support_rep_id
)

SELECT
    s.employee_id 'employee_id',
    e.first_name || ' ' || e.last_name 'employee_name',
    s.supervisor_first_name || ' ' || s.supervisor_last_name 'supervisor',
    e.hire_date 'hire_date',
    et.total 'total_sales'
FROM employee e
LEFT JOIN supervising s ON s.employee_id = e.employee_id
INNER JOIN employee_totals et ON et.employee_id = e.employee_id
    


 * sqlite:///Datasets/chinook.db
Done.


employee_id,employee_name,supervisor,hire_date,total_sales
3,Jane Peacock,Nancy Edwards,2017-04-01 00:00:00,1731.51
4,Margaret Park,Nancy Edwards,2017-05-03 00:00:00,1584.0
5,Steve Johnson,Nancy Edwards,2017-10-17 00:00:00,1393.92


The output of this query reveals that Jane Peacock has garnered the most in total sales, however it is important to note how long each sales support agent has been working for Chinook, as Jane Peacock has also worked there for the longest. In contrast, although Steve Johnson has ~20% less in total sales than Jane, he has worked there for 6.5 months less than Jane. 

## Analysing Sales by Country

In [102]:
%%sql

WITH
    countries_others AS(
        SELECT
            CASE
                WHEN (SELECT COUNT(*) FROM customer c2
                      WHERE c2.country = c1.country
                      ) = 1 THEN 'Other'
                ELSE c1.country
            END AS country,
            c1.customer_id
        FROM customer c1
),
    
    country_count AS(
        SELECT country,
        COUNT(*) 'count',
        CASE
            WHEN country = 'Other' THEN 1
            ELSE 0
        END AS rank
        FROM countries_others
        GROUP BY country
)

    
SELECT
    co.country,
    cc.count 'total_customers',
    ROUND(SUM(i.total), 2) 'total_sales',
    ROUND(SUM(i.total) / cc.count, 2) 'avg_sale_per_customer'
FROM countries_others co
INNER JOIN country_count cc ON cc.country = co.country
INNER JOIN invoice i ON i.customer_id = co.customer_id
GROUP BY 1
ORDER BY cc.rank, total_sales DESC

 * sqlite:///Datasets/chinook.db
Done.


country,total_customers,total_sales,avg_sale_per_customer
USA,13,1040.49,80.04
Canada,8,535.59,66.95
Brazil,5,427.68,85.54
France,5,389.07,77.81
Germany,4,334.62,83.66
Czech Republic,2,273.24,136.62
United Kingdom,3,245.52,81.84
Portugal,2,185.13,92.57
India,2,183.15,91.58
Other,15,1094.94,73.0


## Albums vs Individual Tracks
The Chinook store is setup in a way that allows customers to make purchases in one of two ways:
* Purchase a whole album outright
* Purchase a collection of one or more individual tracks

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums, they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

I will be finding out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

There are two edge cases that must be highlighted:
* Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
* Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since my analysis is concerned with maximizing revenue, albums consisting of only a few tracks can safely be ignored. The company has previously completed analysis to confirm that the second case does not happen often, so I will also ignore this case.

In [136]:
%%sql

CREATE VIEW invoice_album AS
SELECT
    il.invoice_line_id,
    il.invoice_id,
    il.track_id,
    t.album_id,
    t.name 'track',
    a.title 'album'
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN album a ON a.album_id = t.album_id

 * sqlite:///Datasets/chinook.db
Done.


[]

In [121]:
%%sql

CREATE VIEW album_track AS
SELECT * FROM track t
INNER JOIN album a ON a.album_id = t.album_id

 * sqlite:///Datasets/chinook.db
Done.


[]

In [122]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///Datasets/chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [194]:
%%sql

WITH album_checker AS(
    SELECT 
        invoice_id,
        COUNT(invoice_id) 'invoice_count',
        COUNT(DISTINCT(album_id)) 'album_count'
    FROM invoice_album
    GROUP BY invoice_id
),

    album_track AS(
        SELECT
            ia.*,
            CASE
                WHEN (ac.invoice_count > 3 AND ac.album_count = 1) THEN 'album'
                ELSE 'track'
            END AS 'purchase_type'
        FROM invoice_album ia
        INNER JOIN album_checker ac ON ac.invoice_id = ia.invoice_id
),

    purchase_type_totals AS(
        SELECT
            purchase_type,
            COUNT(purchase_type) 'total_purchases'
        FROM (SELECT invoice_id, purchase_type
              FROM album_track
              GROUP BY 1)
        GROUP BY purchase_type
)

SELECT
    *,
    CAST(total_purchases AS Float) / (SELECT CAST(SUM(total_purchases) AS Float) FROM purchase_type_totals) 'purchase_percentage'
FROM purchase_type_totals

 * sqlite:///Datasets/chinook.db
Done.


purchase_type,total_purchases,purchase_percentage
album,132,0.21498371335504887
track,482,0.7850162866449512


Here, we can see that album purchases equate to ~21% of all Chinook sales. Therefore, it is recommended that management do not change their strategy in order to save money as this will result in a significant loss in sales.

### Correct SQL Query

In [200]:
%%sql
WITH invoice_first_track AS (
  SELECT
    il.invoice_id AS invoice_id,
    MIN(il.track_id) AS first_track_id
  FROM
    invoice_line il
  GROUP BY
    1
)

-- Use a subquery to select the results of the invoice_first_track CTE and determine whether customers made album purchases
SELECT
  album_purchase,
  COUNT(invoice_id) AS number_of_invoices,
  CAST(COUNT(invoice_id) AS FLOAT) / (
    SELECT COUNT(*) FROM invoice
  ) AS percent
FROM
  (
    SELECT
      ifs.*,
      CASE
        -- Use the EXCEPT operator to compare the tracks in the first invoice with the tracks in subsequent invoices,
        -- and determine whether any tracks from the album were purchased in subsequent invoices.
        -- If the result of the EXCEPT is NULL, it means that all tracks from the album were purchased in subsequent invoices,
        -- and the customer made an album purchase.
        -- If the result of the EXCEPT is not NULL, it means that at least one track from the album was not purchased in subsequent invoices,
        -- and the customer did not make an album purchase.
        WHEN (
          SELECT
            t.track_id
          FROM
            track t
          WHERE
            t.album_id = (
              SELECT
                t2.album_id
              FROM
                track t2
              WHERE
                t2.track_id = ifs.first_track_id
            )
          EXCEPT
          SELECT
            il2.track_id
          FROM
            invoice_line il2
          WHERE
            il2.invoice_id = ifs.invoice_id
        ) IS NULL
        AND (
          SELECT
            il2.track_id
          FROM
            invoice_line il2
          WHERE
            il2.invoice_id = ifs.invoice_id
          EXCEPT
          SELECT
            t.track_id
          FROM
            track t
          WHERE
            t.album_id = (
              SELECT
                t2.album_id
              FROM
                track t2
              WHERE
                t2.track_id = ifs.first_track_id
            )
        ) IS NULL
        THEN "yes"
        ELSE "no"
      END AS album_purchase
    FROM
      invoice_first_track ifs
  ) subquery
-- Group by album_purchase to get the counts and percentages for each type of purchase
GROUP BY
  album_purchase;

 * sqlite:///Datasets/chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


## Which Artist Is Used in the Most Playlists?

In [6]:
%%sql

CREATE VIEW playlist_data AS
SELECT pt.playlist_id, t.track_id, t.name 'track', ar.name 'artist'FROM
playlist_track pt
INNER JOIN track t ON t.track_id = pt.track_id
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist ar ON ar.artist_id = al.artist_id

 * sqlite:///Datasets/chinook.db
Done.


[]

In [10]:
%%sql

SELECT artist, COUNT(*) FROM playlist_data
GROUP BY 1
ORDER BY 2 DESC

 * sqlite:///Datasets/chinook.db
Done.


artist,COUNT(*)
Iron Maiden,516
U2,333
Metallica,296
Led Zeppelin,252
Deep Purple,226
Lost,184
Pearl Jam,177
Faith No More,145
Eric Clapton,145
Lenny Kravitz,143


Here, we can see that the artists found on playlists most are:
* Iron Maiden
* U2
* Metallica
* Led Zeppelin
* Deep Purple

## Do Protected vs Non-Protected Media Types Have An Effect on Popularity

In [11]:
%%sql

SELECT * FROM media_type

 * sqlite:///Datasets/chinook.db
Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


Chinook uses 5 different audio file formats for its music, however there are 3 core types:
* MPEG
* MPEG-4
* AAC

The variety in audio file formats predominantly comes from whether the file is protected or not. I will assess whether audio files that are protected generate more sales than audio files that are unprotected.

In [17]:
%%sql

WITH track_media AS(
    SELECT t.track_id, t.name 'track', mt.name 'media_type', il.quantity
    FROM track t
    INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
    INNER JOIN invoice_line il ON il.track_id = t.track_id
),


    track_protection AS(
        SELECT
            *,
            CASE
                WHEN media_type LIKE 'Protected%' THEN 'Protected'
                ELSE 'Unprotected'
            END AS protection_type
        FROM track_media
)

SELECT protection_type, COUNT(protection_type)
FROM track_protection
GROUP BY 1

 * sqlite:///Datasets/chinook.db
Done.


protection_type,COUNT(protection_type)
Protected,442
Unprotected,4315


It is clear that unprotected tracks are purchased more by customers compared to protected tracks, with there being 876% increase in sales of unprotected audio files. However, having a greater number of sales may not directly reveal greater popularity in unprotected audio files as we are yet to know how many tracks fall into each protection type, regardless of if they generated sales or not. It may be that there are simply more unprotected tracks available to purchase compared to protected tracks, resulting in the large disparity in sales between the two.

In [19]:
%%sql

WITH track_media AS(
    SELECT t.track_id, t.name 'track', mt.name 'media_type'
    FROM track t
    INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
),

  track_protection AS(
        SELECT
            *,
            CASE
                WHEN media_type LIKE 'Protected%' THEN 'Protected'
                ELSE 'Unprotected'
            END AS protection_type
        FROM track_media
)

SELECT protection_type, COUNT(protection_type)
FROM track_protection
GROUP BY 1

 * sqlite:///Datasets/chinook.db
Done.


protection_type,COUNT(protection_type)
Protected,451
Unprotected,3052


Here, we see the assumption that there are a greater number of unprotected audio files available to customers is true. So, these files generating more total sales should come as no surprise. However, given the values outputted by the two previous queries, protected audio files generate 0.977 average sales per file, and unprotected files generate 1.415 average sales per file. This reveals that unprotected files are bought more frequently than unprotected files and can be an indicator of popularity, however other factors would need to be considered.